
## 5. Penalizaciones L1 y L2
Sin el uso de librerías, programe las penalizaciones L1 y L2, aplicando normalización.


## 1. Preparación del Entorno
Primero, montamos Google Drive y cargamos el dataset:

In [38]:
from google.colab import drive
import pandas as pd

# Montar Google Drive
drive.mount('/content/drive')

# Cargar el archivo CSV
df = pd.read_csv('/content/drive/My Drive/TAREAS U/8vo semestre/IA/Apoyo 1er parcial/4final.csv', delimiter=',')
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   Rank        Date  Danceability    Energy  Loudness  Speechiness  \
0     1  2023-05-29     -0.060460  0.681445 -0.029892    -0.670052   
1     1  2023-05-29     -0.060460  0.681445 -0.029892    -0.670052   
2     2  2023-05-29     -0.181634  0.963016  0.377901    -0.364715   
3     3  2023-05-29      1.030105 -1.188990 -0.206826     2.601422   
4     3  2023-05-29      1.030105 -1.188990 -0.206826     2.601422   

   Acousticness  Instrumentalness   Valence  Points (Total)  ...  \
0      1.047175         -0.171931  1.333454        1.654353  ...   
1      1.047175         -0.171931  1.333454        1.654353  ...   
2     -0.414239          7.023328 -1.240858        1.637321  ...   
3     -0.113359         -0.171931  0.153561        1.620290  ...   
4     -0.113359         -0.171931  0.153561        1.620290  ...   

   Nationality_Unknown  Continent_Africa 

## 2. Dividir los Datos
A continuación, separamos las características relevantes y la variable objetivo. Usaremos Points (Total) como variable objetivo tanto para regresión como para clasificación:

In [39]:
import numpy as np
from sklearn.model_selection import train_test_split

# Selecciona las características relevantes
features = ['Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Valence']
X = df[features]

# Variable objetivo para regresión
y_regression = df['Points (Total)']

# Variable objetivo para clasificación
threshold = df['Points (Total)'].median()
df['Success'] = (df['Points (Total)'] > threshold).astype(int)
y_classification = df['Success']

# Dividir los datos
X_train, X_test, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=0.2, random_state=42)
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X, y_classification, test_size=0.2, random_state=42)


## 3. Evaluación de Modelos
Probamos varios modelos de regresión y clasificación. Evaluamos su desempeño utilizando métricas adecuadas.

In [40]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report

# Lista de modelos para regresión
regression_models = {
    "Regresión Lineal": LinearRegression(),
    "Regresión Lasso": Lasso(),
    "Regresión Ridge": Ridge(),
    "Árbol de Decisión": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

# Evaluar modelos de regresión
for name, model in regression_models.items():
    model.fit(X_train, y_train_reg)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test_reg, y_pred)
    r2 = r2_score(y_test_reg, y_pred)
    print(f"{name} - MSE: {mse}, R^2: {r2}")

# Lista de modelos para clasificación
classification_models = {
    "Regresión Logística": LogisticRegression(max_iter=1000),
    "Árbol de Decisión": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Evaluar modelos de clasificación
for name, model in classification_models.items():
    model.fit(X_train_class, y_train_class)
    y_pred_class = model.predict(X_test_class)
    accuracy = accuracy_score(y_test_class, y_pred_class)
    report = classification_report(y_test_class, y_pred_class)
    print(f"{name} - Accuracy: {accuracy}")
    print(report)


Regresión Lineal - MSE: 0.9816917305547017, R^2: 0.004105394544546281
Regresión Lasso - MSE: 0.9859535483272904, R^2: -0.0002180821610759942
Regresión Ridge - MSE: 0.981691659243298, R^2: 0.004105466887665776
Árbol de Decisión - MSE: 0.18122439545782154, R^2: 0.8161536944888081
Random Forest - MSE: 0.17939147010546327, R^2: 0.8180131381550848
Gradient Boosting - MSE: 0.4061026655488337, R^2: 0.5880219408055531
Regresión Logística - Accuracy: 0.5186543023506578
              precision    recall  f1-score   support

           0       0.52      0.56      0.54      2324
           1       0.52      0.47      0.50      2313

    accuracy                           0.52      4637
   macro avg       0.52      0.52      0.52      4637
weighted avg       0.52      0.52      0.52      4637

Árbol de Decisión - Accuracy: 0.8768600388182014
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      2324
           1       0.89      0.86      0.87      2

## 4. Comparación de Modelos

Después de evaluar los modelos, observamos que:

**Métricas de Regresión:**
- **Árbol de Decisión:** MSE: 0.181, R²: 0.816
- **Random Forest:** MSE: 0.179, R²: 0.818

**Métricas de Clasificación:**
- **Árbol de Decisión:** Accuracy: 0.8766
- **Random Forest:** Accuracy: 0.8782

Dado que Random Forest supera al Árbol de Decisión en ambas métricas, se elige Random Forest como el modelo preferido.


## 5. Selección del Mejor Lambda para Penalización
Para ajustar la penalización L1 (Lasso) y L2 (Ridge), normalizamos los datos y probamos diferentes valores de lambda.

In [41]:
# Normalización
X_mean = np.mean(X.values, axis=0)
X_std = np.std(X.values, axis=0)
X_normalized = (X - X_mean) / X_std

# Inicializar coeficientes
coef = np.zeros(X_normalized.shape[1])

# Parámetros
learning_rate = 0.01
iterations = 1000
lambdas = np.logspace(-3, 3, 7)  # Desde 0.001 hasta 1000

# Resultados
lasso_results = []
ridge_results = []

# Bucle para evaluar Lasso
for alpha in lambdas:
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_normalized, y_regression)
    lasso_pred = lasso_model.predict(X_normalized)
    lasso_mse = mean_squared_error(y_regression, lasso_pred)
    lasso_results.append((alpha, lasso_mse))

# Bucle para evaluar Ridge
for alpha in lambdas:
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_normalized, y_regression)
    ridge_pred = ridge_model.predict(X_normalized)
    ridge_mse = mean_squared_error(y_regression, ridge_pred)
    ridge_results.append((alpha, ridge_mse))

# Imprimir resultados
print("Lasso Results:")
for alpha, mse in lasso_results:
    print(f"Lambda: {alpha:.3f}, MSE: {mse:.3f}")

print("\nRidge Results:")
for alpha, mse in ridge_results:
    print(f"Lambda: {alpha:.3f}, MSE: {mse:.3f}")


Lasso Results:
Lambda: 0.001, MSE: 0.994
Lambda: 0.010, MSE: 0.995
Lambda: 0.100, MSE: 1.000
Lambda: 1.000, MSE: 1.000
Lambda: 10.000, MSE: 1.000
Lambda: 100.000, MSE: 1.000
Lambda: 1000.000, MSE: 1.000

Ridge Results:
Lambda: 0.001, MSE: 0.994
Lambda: 0.010, MSE: 0.994
Lambda: 0.100, MSE: 0.994
Lambda: 1.000, MSE: 0.994
Lambda: 10.000, MSE: 0.994
Lambda: 100.000, MSE: 0.994
Lambda: 1000.000, MSE: 0.994


## 6. Conclusión
Analizando los resultados de la penalización, encontramos que:

Ridge mantiene un MSE más estable en comparación con Lasso.
Se recomienda usar Ridge con
𝜆
=
0.001
λ=0.001 o
𝜆
=
0.010
λ=0.010 para un buen balance entre regularización y precisión.